In [40]:
import matplotlib.pylab as plt
import numpy as np

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub

import os
import pandas as pd
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, Callback


# Set memory growth for GPU devices
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)


print("TF version:", tf.__version__)
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

TF version: 2.13.0
Hub version: 0.14.0
GPU is available


In [41]:
train_directory='../input/100-bird-species/train'
val_directory='../input/100-bird-species/valid'
test_directory='../input/100-bird-species/test'


train_datagen=ImageDataGenerator(
 rescale=1/255.0, 
 rotation_range=10, 
 zoom_range=0.05, 
 width_shift_range=0.05, 
 height_shift_range=0.05, 
 shear_range=0.05,
 horizontal_flip=True,
 fill_mode='nearest')

batch_size =64

train_generator = train_datagen.flow_from_directory(
    directory=train_directory,
    target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode='categorical', 
    shuffle=True, 
    seed=42
)
val_datagen=ImageDataGenerator(rescale=1/255.0)
valid_generator = val_datagen.flow_from_directory(
    directory=val_directory,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=True,
    seed=42
)
test_datagen = ImageDataGenerator(rescale=1/255.0) 
test_generator = test_datagen.flow_from_directory(
  directory=test_directory,
  target_size=(224,224),
    color_mode='rgb',
    batch_size=batch_size,
    class_mode="categorical",
    shuffle=False,
    seed=42
)

Found 84635 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.
Found 2625 images belonging to 525 classes.


In [42]:

# Training
steps_per_epoch=train_generator.n//train_generator.batch_size
validation_steps=valid_generator.n//valid_generator.batch_size
epochs = 50
lr = 0.0001

# Create the model
model = tf.keras.Sequential([
    # Pre-trained ResNet-50 model
    hub.KerasLayer("https://www.kaggle.com/models/tensorflow/resnet-50/frameworks/TensorFlow2/variations/feature-vector/versions/1", trainable=False),
    tf.keras.layers.Flatten(),

    # Additional layers
    # tf.keras.layers.Dropout(rate=0.2),
    tf.keras.layers.Dense(1024, activation='relu'),
    # tf.keras.layers.Dropout(rate=0.2),
    # tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dropout(rate=0.3),
    # Output Dense layer
    tf.keras.layers.Dense(525,  activation= 'softmax')
])



model.build([None, 224, 224, 3])
model.compile(loss='categorical_crossentropy',optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=lr),metrics=['acc'])

model.summary()

Attaching model 'tensorflow/resnet-50/tensorflow2/feature-vector/1' to your Kaggle notebook...


Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer_8 (KerasLayer)  (None, 2048)              23561152  
                                                                 
 flatten_1 (Flatten)         (None, 2048)              0         
                                                                 
 dense_15 (Dense)            (None, 1024)              2098176   
                                                                 
 dropout_7 (Dropout)         (None, 1024)              0         
                                                                 
 dense_16 (Dense)            (None, 525)               538125    
                                                                 
Total params: 26197453 (99.94 MB)
Trainable params: 2636301 (10.06 MB)
Non-trainable params: 23561152 (89.88 MB)
_________________________________________________________________


In [43]:
%load_ext tensorboard
tensorboard_callback = TensorBoard(log_dir=f'/kaggle/working/logs', histogram_freq=1)

# To visualize TensorBoard in the Colab notebook
%tensorboard --logdir {root}logs


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# Define callbacks


class CustomCallback(Callback):
    def on_epoch_end(self, epoch, logs=None):
        if (epoch + 1) % 5 == 0:
            val_acc = self.model.evaluate(test_generator, return_dict=True)['acc']
            print(f'test accuracy at epoch {epoch + 1}: {val_acc}')

            self.model.save(f'/kaggle/working/models/model_val_acc_{epoch + 1}.h5')

custom_callback = CustomCallback()




model.fit(
    train_generator,
    epochs=epochs,
    steps_per_epoch=steps_per_epoch,
    validation_data=valid_generator,
    validation_steps=validation_steps,
    callbacks=[tensorboard_callback, custom_callback]
)



Epoch 1/50
1322/1322 [==============================] - 1087s 817ms/step - loss: 2.7123 - acc: 0.4554 - val_loss: 0.8317 - val_acc: 0.8552
Epoch 2/50
1322/1322 [==============================] - 1068s 808ms/step - loss: 0.9687 - acc: 0.7656 - val_loss: 0.4884 - val_acc: 0.8990
Epoch 3/50
1322/1322 [==============================] - 1053s 796ms/step - loss: 0.6701 - acc: 0.8327 - val_loss: 0.3552 - val_acc: 0.9200
Epoch 4/50
1322/1322 [==============================] - 1024s 775ms/step - loss: 0.5248 - acc: 0.8645 - val_loss: 0.2988 - val_acc: 0.9272
Epoch 5/50
42/42 [==============================] - 9s 213ms/step - loss: 0.1986 - acc: 0.9543
test accuracy at epoch 5: 0.954285740852356


/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


1322/1322 [==============================] - 1048s 792ms/step - loss: 0.4351 - acc: 0.8873 - val_loss: 0.2541 - val_acc: 0.9386
Epoch 6/50
1322/1322 [==============================] - 1050s 794ms/step - loss: 0.3715 - acc: 0.9020 - val_loss: 0.2400 - val_acc: 0.9345
Epoch 7/50
1322/1322 [==============================] - 1049s 793ms/step - loss: 0.3254 - acc: 0.9128 - val_loss: 0.2163 - val_acc: 0.9398
Epoch 8/50
1322/1322 [==============================] - 1031s 780ms/step - loss: 0.2858 - acc: 0.9240 - val_loss: 0.1971 - val_acc: 0.9451
Epoch 9/50
1322/1322 [==============================] - 1056s 799ms/step - loss: 0.2589 - acc: 0.9299 - val_loss: 0.1922 - val_acc: 0.9447
Epoch 10/50
42/42 [==============================] - 10s 222ms/step - loss: 0.1211 - acc: 0.9680
test accuracy at epoch 10: 0.9679999947547913
1322/1322 [==============================] - 1049s 793ms/step - loss: 0.2321 - acc: 0.9368 - val_loss: 0.1848 - val_acc: 0.9489
Epoch 11/50
1322/1322 [======================

In [ ]:
results = model.evaluate(test_generator, verbose=0)

print("    Test Loss: {:.5f}".format(results[0]))
print("Test Accuracy: {:.2f}%".format(results[1] * 100))